# Automate Event Planning

In [13]:
import os
import warnings
from crewai import Task, Agent, Crew
import utils
from dotenv import load_dotenv

warnings.filterwarnings('ignore')
load_dotenv()

openai_api_key = utils.get_openai_api_key()
SERPER_API_KEY = utils.get_serper_api_key()
OPENAI_MODEL_NAME = os.getenv('OPENAI_MODEL_NAME')

In [14]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

# Creating Agents

In [15]:
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
         "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [16]:
# Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [17]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

# Creating Venue Pydantic Object

In [18]:
from pydantic import BaseModel


class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

# Creating Tasks

In [19]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",
    # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

In [20]:
logistics_task = Task(
    description="Coordinate catering and "
                "equipment for an event "
                "with {expected_participants} participants "
                "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [21]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

# Creating the Crew

In [22]:
event_management_crew = Crew(
    agents=[venue_coordinator,
            logistics_manager,
            marketing_communications_agent],

    tasks=[venue_task,
           logistics_task,
           marketing_task],

    verbose=True
)

2024-08-12 15:29:10,143 - 8496200384 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


In [23]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

In [24]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
I should start by gathering all necessary information about potential venues in San Francisco that could accommodate a Tech Innovation Conference. This includes details like size, amenities, cost, and availability. I should begin by searching the internet for suitable venues.

Action: Search the internet
Action Input: {"search_query": "best venues for tech conferences in San Francisco"} 


Search results: Title: Top 12 tech conferences in San Francisco for 2024 - TravelPerk
Link: https://www.travelperk.com/blog/top-tech-conferences-in-san-francisco/
Snippet: Top 12 tech conferences in San Francisco for 2024 · Developer Week · Women Impact Tech · SaaS Connect · Cybersecurity and Cloud Congress · Startup ...
---
Title: The Ultimate Calendar of 50 Best Tech Conferences in San Francisco
Link: h

In [25]:
import json
from pprint import pprint

with open('venue_details.json') as f:
    data = json.load(f)

pprint(data)

{'address': 'SoMa, San Francisco',
 'booking_status': 'TBD',
 'capacity': 2000,
 'name': 'City View at METREON'}


In [27]:
from IPython.display import Markdown

Markdown("marketing_report.md")

# Marketing and Attendee Engagement Report for Tech Innovation Conference

## Venue Details

**Name:** City View at METREON  
**Address:** SoMa, San Francisco  
**Capacity:** 2000  
**Booking Status:** TBD (confirmation in progress)

### Venue Features
- **Breathtaking Views:** The venue offers stunning views of the San Francisco skyline.
- **Event Space:** 31,000 sq. ft. of flexible event space, including an 11,000 sq. ft. open-air terrace.
- **Interior Design:** Features floor-to-ceiling glass windows, seven lush living walls, and a luxe remodeled interior.
- **Amenities:** Expansive catering space, robust power, WiFi, and data capabilities, motorized truss system, and permanent tent anchors.
- **Accessibility:** Centrally located with proximity to Moscone Center, public parking, transit, and six flagship hotels within a block.

## Marketing Strategy

### Goals
- Engage at least 500 potential attendees.

### Target Audience
- Tech enthusiasts
- Industry professionals
- Entrepreneurs
- Investors
- Students and academics interested in technology and innovation

### Marketing Channels
1. **Website and Blog:**
   - Create a dedicated event page with detailed information, registration forms, and engaging content.
   - Publish blog posts highlighting key speakers, panel discussions, and innovative sessions.

2. **Email Marketing:**
   - Send out personalized invitations to industry professionals, past attendees, and mailing list subscribers.
   - Regularly update registered participants with event news, speaker announcements, and reminders.

3. **Social Media:**
   - Leverage platforms like LinkedIn, Twitter, Facebook, and Instagram to create buzz.
   - Share engaging content, including speaker bios, behind-the-scenes preparations, and interactive polls.
   - Use event-specific hashtags to increase visibility and encourage attendee interaction.

4. **Press Releases:**
   - Distribute press releases to tech news outlets and local media to reach a wider audience.
   - Highlight unique aspects of the conference, such as keynote speakers, innovative sessions, and networking opportunities.

5. **Partnerships and Sponsorships:**
   - Collaborate with tech companies, startups, and industry associations to co-promote the event.
   - Offer sponsorship packages with benefits like logo placement, speaking opportunities, and exhibition booths.

6. **Online Advertising:**
   - Run targeted ads on Google, LinkedIn, and Facebook to reach a broader audience.
   - Utilize retargeting campaigns to re-engage visitors who showed interest but did not register.

### Engagement Activities
- **Interactive Sessions:** Include Q&A sessions, live polls, and interactive workshops to keep attendees engaged.
- **Networking Opportunities:** Set up virtual and in-person networking sessions to facilitate connections among attendees.
- **Contests and Giveaways:** Organize contests and giveaways on social media to generate excitement and increase participation.

## Next Steps
- Await confirmation of the booking status from the Venue Coordinator.
- Finalize the event page and initial marketing materials.
- Launch the email marketing campaign and social media promotions.
- Continue to monitor and adjust the marketing strategy based on engagement metrics.

By implementing this comprehensive marketing strategy, we aim to effectively promote the Tech Innovation Conference and engage at least 500 potential attendees.